In [1]:
import urllib

import numpy as np
import tensorflow as tf
tf.enable_v2_behavior()

from rcnn_sat import preprocess_image, bl_net

# Restore model and extract activations
This example shows how to extract activations from a pre-trained model. We'll use the BL model trained on eco-set.

## Building the model

First, we build the graph for the model using a placeholder for the input.

In [2]:
input_layer = tf.keras.layers.Input((128, 128, 3))
model = bl_net(input_layer, classes=565, cumulative_readout=True)

Instructions for updating:
Colocations handled automatically by placer.


All model functions take an input layer as the first argument and the number of classes as the second. Note for ecoset `classes=565` and for ImageNet `classes=1000`.

BL has an additional argument specifying whether or not to use a cumulative readout. By deafult `cumulative_readout=False`.

## Loading the weights
Download the pre-trained weights for the models from OSF (weights for other models can be found [here](https://osf.io/mz9hw/)).

In [ ]:
_, msg = urllib.request.urlretrieve(
    'https://osf.io/9td5p/download', 'bl_ecoset.h5')
print(msg)

Load the weights into the model

In [3]:
model.load_weights('bl_ecoset.h5')

## Extract activations

To get the activation in Keras we need to know the name of the layer. The name of the output at each layer and time step has the format `ReLU_Layer_{layer_index}_Time_{time_index}`. So, the output of the first layer on the third time step is labelled `ReLU_Layer_0_Time_2` (note that zero-indexing is used).

We use this to create a function mapping from input to output that we call to get a numpy array of the layer activations.

In [4]:
get_layer_activation = tf.keras.backend.function(
    [model.input],
    [model.get_layer('ReLU_Layer_0_Time_2').output])

Now we'll get the activations based on a random image. We scale the image into the correct range for the pre-trained networks by passing the image through `preprocess_image`. The image must be in `uint8` format (values in the range `[0, 255]`).

In [5]:
random_img = np.random.randint(0, 256, (1, 128, 128, 3), dtype=np.uint8)
preprocessed_img = preprocess_image(random_img)
get_layer_activation(preprocessed_img)

[array([[[[1.7345557 , 1.2931564 , 0.        , ..., 0.40440372,
           0.55534697, 1.5363963 ],
          [0.07397947, 1.6641147 , 0.        , ..., 0.        ,
           0.        , 1.4803685 ],
          [0.        , 0.06762171, 0.        , ..., 0.        ,
           0.        , 0.7741227 ],
          ...,
          [0.        , 0.2210756 , 0.50375617, ..., 0.        ,
           0.        , 0.        ],
          [3.1864197 , 0.21792549, 0.        , ..., 0.        ,
           0.6976775 , 0.        ],
          [0.        , 0.1745987 , 0.204889  , ..., 0.        ,
           1.101606  , 0.        ]],
 
         [[0.        , 0.66232175, 0.        , ..., 0.        ,
           0.        , 1.3742282 ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.59119904],
          [0.        , 0.        , 1.1477087 , ..., 0.        ,
           0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.4341197 , ..., 0.        ,
   